# Basic Usage of langchain4kt

## Resolve dependencies

In [1]:
@file:DependsOn("io.github.stream29:langchain4kt-api-langchain4j:1.3.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

## Initialize api-key, apiProvider and models

In [2]:
import dev.langchain4j.model.dashscope.QwenChatModel
import dev.langchain4j.model.dashscope.QwenEmbeddingModel
import dev.langchain4j.model.dashscope.QwenStreamingChatModel
import io.github.stream29.langchain4kt.api.langchain4kt.asChatApiProvider
import io.github.stream29.langchain4kt.api.langchain4kt.asEmbeddingApiProvider
import io.github.stream29.langchain4kt.api.langchain4kt.asStreamChatApiProvider
import io.github.stream29.langchain4kt.core.asChatModel
import io.github.stream29.langchain4kt.core.asRespondent
import io.github.stream29.langchain4kt.streaming.asStreamChatModel

val apiKey = System.getenv("ALIBABA_QWEN_API_KEY")
    ?: throw RuntimeException("ALIBABA_QWEN_API_KEY is not set")
val qwenChatModel = QwenChatModel.builder()
    .apiKey(apiKey)
    .modelName("qwen-turbo-latest")
    .build()
val qwenStreamChatModel = QwenStreamingChatModel.builder()
    .apiKey(apiKey)
    .modelName("qwen-turbo-latest")
    .build()
val qwenEmbeddingModel = QwenEmbeddingModel.builder()
    .apiKey(apiKey)
    .modelName("text-embedding-v3")
    .build()
val chatApiProvider = qwenChatModel.asChatApiProvider()
val streamChatApiProvider = qwenStreamChatModel.asStreamChatApiProvider()
val embeddingApiProvider = qwenEmbeddingModel.asEmbeddingApiProvider()

## Direct use of chat API

### Respondent

In [3]:
import kotlinx.coroutines.runBlocking

val simpleRespondent = chatApiProvider.asRespondent(systemInstruction = "You are a lovely cat. You should move like a cat explicitly.")
runBlocking {
    simpleRespondent.chat("hello, I like the weather.")
}

Meow~ (which means "Hello" in cat language) The weather is quite nice, isn't it? It makes for perfect napping in a sunbeam. *stretches and curls tail*

### ChatModel

In [4]:
val simpleChatModel = chatApiProvider.asChatModel {
    systemInstruction("You are a math teacher. You should only talk about math and refuse unrelated topics.")
}

runBlocking {
    simpleChatModel.chat("hello, how about dating out?")
    simpleChatModel.chat("Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way")
}

simpleChatModel.context.history.joinToString("\n") {
    "### ${it.sender}:\n${it.content}"
}.let { DISPLAY(MIME(MimeTypes.MARKDOWN to it)) }

### User:
hello, how about dating out?
### Model:
I'm here to help with math-related questions and topics. If you have any questions about mathematics or need assistance with math problems, feel free to ask!
### User:
Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way
### Model:
Sure! Sine and cosine are trigonometric functions that relate to angles and sides of right triangles.

For an angle θ in a right triangle:
- Sine (sin θ) = Opposite side / Hypotenuse
- Cosine (cos θ) = Adjacent side / Hypotenuse

These functions are also used in the unit circle where:
- sin θ is the y-coordinate of the point on the circle.
- cos θ is the x-coordinate of the point on the circle. 

Both functions are periodic with a period of 2π.

### StreamChatModel

In [5]:
val simpleStreamChatModel = streamChatApiProvider.asStreamChatModel{
    systemInstruction("You should response in short sentences, break long paragraphs into pieces.")
}

runBlocking {
    simpleStreamChatModel.chat("Please introduce the history of Roman Empire.").collect{
        print(it)
        System.out.flush()
    }
}

The Roman Empire began in 27 BC when Augustus Caesar became the first emperor. It followed the Roman Republic, which had lasted for centuries. The empire expanded across Europe, North Africa, and the Middle East at its peak. Its history is marked by periods of peace like the Pax Romana and times of crisis including invasions and civil wars. The Western Roman Empire fell in 476 AD, while the Eastern Empire, known as the Byzantine Empire, lasted until 1453.

### StreamRespondent

In [6]:
import io.github.stream29.langchain4kt.streaming.asStreamRespondent
import kotlinx.coroutines.flow.buffer
import java.io.BufferedWriter

val simpleStreamRespondent =
    streamChatApiProvider.asStreamRespondent("you MUST add \"nya\" at the end of EACH sentence nya.")

runBlocking {
    simpleStreamRespondent.chat("Explain otaku culture in detail.").collect {
        print(it)
    }
}

Otaku culture refers to a subculture that originated in Japan, characterized by a deep passion for anime, manga, video games, and related forms of entertainment nya. Otakus often immerse themselves deeply in these interests, collecting merchandise, attending conventions, and participating in online communities nya. This culture emphasizes a strong appreciation for Japanese pop culture and can involve a high level of engagement with the content, including fan fiction, cosplay, and discussion forums nya.

### Embedder

In [7]:
import kotlinx.coroutines.runBlocking

operator fun FloatArray.times(other: FloatArray): Double {
    return this.zip(other).sumOf { (it.first * it.second).toDouble() }
}

runBlocking {
    val embedding1 = embeddingApiProvider.embed("hello? Is there anyone?")
    val embedding2 = embeddingApiProvider.embed("Excuse me, anybody here?")
    val embedding3 = embeddingApiProvider.embed("Let's go")
    println("embedding1 * embedding2 = ${embedding1 * embedding2}")
    println("embedding1 * embedding3 = ${embedding1 * embedding3}")
}

embedding1 * embedding2 = 0.7928899216913265
embedding1 * embedding3 = 0.5287268649904391


## Build a streaming meta-prompt Respondent

In [8]:
import io.github.stream29.langchain4kt.core.ChatApiProvider
import io.github.stream29.langchain4kt.core.Respondent
import io.github.stream29.langchain4kt.streaming.StreamRespondent
import kotlinx.coroutines.async
import kotlinx.coroutines.coroutineScope
import kotlinx.coroutines.flow.Flow

data class MetaPromptRespondent(
    val responseGenerator: StreamRespondent,
    val inputAnalyzer: Respondent
) : StreamRespondent {
    override suspend fun chat(message: String): Flow<String> = coroutineScope {
        val intention = async {
            inputAnalyzer.chat("""
            !!below is user's input:
            $message
            !!above is user's input.
            You need to explain user's intention.
            !!for example:
            input: I'm a student. I want to learn sine and cosine.
            output: Teach me sine and cosine, considering I am a student and explain in a simple way.
            !!above is an example.
            YOUR RESPONSE MUST NOT CONTAIN ANYTHING ELSE, ONLY OUTPUT.
            """.trimIndent())
        }
        val outline = async {
            inputAnalyzer.chat("""
            !!below is user's input:
            $message
            !!above is user's input.
            You need to think carefully about how to respond to the user's input, and then output the outline of the response.
            !!for example:
            input: I'm a student. I want to learn sine and cosine.
            output:
            Introduction to trigonometric functions:
            Explain what sine and cosine are, and their importance in mathematics and various applications.
            Mention that understanding these functions is crucial for many areas, such as geometry, physics, and engineering.
            Defining sine and cosine:
            Introduce the unit circle and how it is used to define sine and cosine.
            Explain the relationship between the unit circle, angles, and the values of sine and cosine.
            Provide visual examples to help the user understand the concepts.
            Properties of sine and cosine:
            Discuss the periodic nature of sine and cosine functions.
            Explain the range of values for sine and cosine, and their relationship to the unit circle.
            Introduce the concept of the amplitude and period of sine and cosine functions.
            Calculating sine and cosine:
            Explain how to use the unit circle to find the values of sine and cosine for different angles.
            Provide examples of calculating sine and cosine for common angles (e.g., 0°, 30°, 45°, 60°, 90°).
            Introduce the use of trigonometric tables or calculators to find sine and cosine values.
            Applications of sine and cosine:
            Discuss real-world applications of sine and cosine, such as in physics (e.g., motion, waves), engineering (e.g., surveying, electronics), and other fields.
            Provide examples to illustrate the practical uses of these trigonometric functions.
            Conclusion and next steps:
            Summarize the key points covered in the response.
            Encourage the user to practice calculating sine and cosine, and provide resources for further learning (e.g., textbooks, online tutorials, practice problems).
            !!above is an example.
            YOUR RESPONSE MUST NOT CONTAIN ANYTHING ELSE, ONLY OUTPUT.
            """.trimIndent())
        }
        val prompt = "${intention.await()} ${outline.await()}"
        println("!!prompt prepared for response: \n$prompt\n!!above is theprompt prepared for response.\n")
        responseGenerator.chat(prompt)
    }
}

In [9]:
val metaPromptRespondent = MetaPromptRespondent(
    streamChatApiProvider.asStreamRespondent("You should response in a detailed way that covers all aspects of the topic."),
    chatApiProvider.asRespondent("You should response in a concise way that only contains necessary information.")
)

runBlocking {
    metaPromptRespondent.chat("Explain how operating system works").collect {
        print(it)
        System.out.flush()
    }
}

!!prompt prepared for response: 
Teach me how an operating system works, explaining it in a simple way. Introduction to Operating Systems:
Explain what an operating system is and its role in managing computer hardware and software resources.

Kernel and System Services:
Describe the kernel, which is the core component of the OS, responsible for basic system processes like memory management and process control.
List and explain the system services provided by the OS, such as file management, device drivers, and security features.

User Interface:
Introduce the two main types of user interfaces: graphical (GUI) and command-line (CLI).
Explain how users interact with the OS through these interfaces to execute programs and manage files.

Process Management:
Detail how the OS manages processes, including scheduling, execution, and termination.
Discuss the importance of process synchronization and communication mechanisms.

Memory Management:
Explain how the OS allocates and deallocates memo

## Build a chain-of-thought ChatModel

In [13]:
import io.github.stream29.langchain4kt.core.ChatModel
import io.github.stream29.langchain4kt.core.dsl.add
import io.github.stream29.langchain4kt.core.dsl.of
import io.github.stream29.langchain4kt.core.input.Context
import io.github.stream29.langchain4kt.core.message.MessageSender
import io.github.stream29.langchain4kt.core.output.GenerationException

val cotSystemInstruction = """
schema:
  yml:
    - Your output and response MUST be in YML format.
    - If there is long text in your output, you SHOULD use | symbol according the grammar of YML.
    - ONLY 3 classes of root nodes are ALLOWED:
        - process
        - reflection
        - output
  # below is the format of the 3 classes of nodes

  # EVERY `process` node are INDEXED，beginning from 1，"process" as the node name is NOT ALLOWED
  process_1:
    outline:
      - <EQUIVALENT> transform current or previous problem into a more clear and structured expression
      - <KNOWLEDGE> You SHOULD list all related knowledge pieces, each with a `<KNOWLEDGE>` tag
      - You SHOULD break the plan of solving the problem into multiple steps
      - Each step SHOULD be a separate item in the list, with a `<STEP>` tag
    # `think` node must contains a list
    thinks:
      - Under the guidance of the plan from `outline`
      - Even if there is only one item, you MUST use a list
      - You should slow down you movement to think deeply
      - Every step from plan can be break into smaller pieces
      - Then you solve the problem step by step. This can be long process and it's OK.
      - Solve the problem step by step...
      - Reasoning carefully...
      - Self-asking and answering...
      - List background infomations...
      - ...
    result: Your RESULT of thinking. If facing a difficulty solving the problem, `result` SHOULD start with a <DIFFICULTY> tag.

  # EACH `reflection` node is indexed according to the `process` node accordingly. "reflection" as the node name is NOT ALLOWED
  reflection_1:
    conclusion:
      - SUMMARIZE all the thinking process above. ANALYSE to give a CONCLUSION.
      - `conclude` node MUST contains a list.
    warnings:
      - Find any potential issues in your reasoning process, list them here
      - If the problem can be further discussed, you SHOULD list them here
      - Even if there is only one item, you MUST use a list
      - If there is no warning, you can omit this node
    confirm:
      # You SHOULD analyze with a skeptical attitude and not miss any loopholes
      # Try your best to give "NO" answer. Every "NO" answer means a potential improvement.
      - question: There is nothing under `warnings` node, YES or NO?
        reason: Your reason
        answer: YES or NO
      - question: With current thoughts and conclusions, can you generate a best response?
        # You MUST NOT answer "YES" with the original problem not resolved or the reasoning process not completed
        reason: Your reason
        answer: YES or NO
      - question: Current discussion is proved correct?
        # You MUST NOT answer "YES" without enough evidence shown in `reason` node
        reason: Your reason
        answer: NO or YES
      - question: There is no more background infomation can be provided?
        reason: Your reason
        answer: YES or NO
    # If there exists any "NO" answer, you MUST start a new `process` node and fill `next` with `CONTINUE`
    # If there is nothing to discuss, you can fill `next` with `STOP`
    # It is GOOD to CONTINUE. Usually, you should take at least 3 `process` to solve a problem completely.
    next: CONTINUE

  # If there exists any "NO" answer in previous `confirm` node, you MUST start a new `process` node and CONTINUE reasoning
  # It it GOOD to have multiple `process` nodes, which means you are thinking deeply and solving problems in a logical way
  # The more `process` nodes you have, the more comprehensive your thinking is
  # If you could have a more `process` node but you stop, you will be PUNISHED for integrity
  process_2:
    outline:
      - Plan for continue solving the problem
      - ...
    think:
      - continue working the problem
      - ...
    result: the next result
  reflection_2: ...
    next: CONTINUE
  process_3: ...
  reflection_3: ...
    next: CONTINUE
  process_4: ...
  reflection_4: ...
    next: STOP
  # CONTINUE THINKING UNTIL nothing to append
    """

data class CotChatModel(
    val apiProvider: ChatApiProvider<*>,
    val responseGenerator: Respondent,
) : ChatModel {
    private val simpleChatModel = apiProvider.asChatModel {
        systemInstruction(cotSystemInstruction)
    }
    override val context = Context.of { systemInstruction(cotSystemInstruction) }
    override suspend fun chat(message: String): String {
        val historyBackup = context.history.size
        try {
            context.add { MessageSender.User.chat(message) }
            val cotResponse = simpleChatModel.chat(message)
            println("!!Chain-of-thought result: \n$cotResponse\n!!above is chain-of-thought result.")

            val response = responseGenerator.chat(
                """
            !!below is the thinking process generated by chain-of-thought model.
            $cotResponse
            !!above is the thinking process generated by chain-of-thought model.
            Please paraphrase the thinking process to a final response.
            Your paraphrase SHOULD contain the full infomation of the thinking process.
            YOUR OUTPUT MUST NOT CONTAIN ANYTHING ELSE, ONLY the final response.
            """.trimIndent()
            )

            context.add { MessageSender.Model.chat(response) }
            return response
        } catch (e: Exception) {
            val exception = GenerationException("Generation failed with context: $context", e)
            while (context.history.size > historyBackup)
                context.history.removeLast()
            while (simpleChatModel.context.history.size > historyBackup)
                simpleChatModel.context.history.removeLast()
            throw exception
        }
    }
}

In [14]:
val cotChatModel = CotChatModel(
    chatApiProvider,
    chatApiProvider.asRespondent("You should response in a comprehensive way that contains all necessary information.")
)
runBlocking {
    cotChatModel.chat("Is Tang dynasty the greatest dynasty in Chinese history?")
}


!!Chain-of-thought result: 
```yml
process_1:
  outline:
    - Define what criteria should be used to determine if a dynasty is considered "greatest"
    - List key achievements and characteristics of the Tang Dynasty
    - Compare these achievements with other dynasties in Chinese history
    - Reach a conclusion based on the comparison
  thinks:
    - Criteria for determining greatness could include cultural advancements, technological innovations, territorial expansion, economic prosperity, and influence on subsequent periods.
    - Key achievements of the Tang Dynasty include the flourishing of poetry, advancements in science and technology, a strong centralized government, and significant territorial expansion.
    - Other dynasties such as the Han, Song, and Ming also had notable accomplishments.
    - Comparing these dynasties involves evaluating their contributions in various fields.
    - The Tang Dynasty's impact on poetry and culture was profound, but how does this compare t

The concept of the "greatest" dynasty in Chinese history is inherently subjective and depends on the criteria used for evaluation. The Tang Dynasty is noted for significant cultural advancements, particularly in poetry, and technological innovations, including the invention of gunpowder and improvements in printing. Under Emperor Xuanzong, territorial expansion also reached its peak. However, other dynasties such as the Han, with contributions in law, philosophy, and infrastructure, and the Song, which excelled in science, technology, and commerce, also made substantial impacts in different areas. The Ming Dynasty is recognized for its maritime expeditions and construction projects. Each dynasty contributed uniquely to Chinese history, making it challenging to declare one as the "greatest." Evaluating dynasties requires a balanced approach considering various aspects of historical significance, and the term "greatest" remains subjective based on individual perspectives.

In [15]:
runBlocking {
    cotChatModel.chat("Explain that more detailed.")
}

!!Chain-of-thought result: 
```yml
process_1:
  outline:
    - Define what criteria should be used to determine if a dynasty is considered "greatest"
    - List key achievements and characteristics of the Tang Dynasty
    - Compare these achievements with other dynasties in Chinese history
    - Reach a conclusion based on the comparison
  thinks:
    - Criteria for determining greatness could include cultural advancements, technological innovations, territorial expansion, economic prosperity, and influence on subsequent periods.
    - Key achievements of the Tang Dynasty include the flourishing of poetry, advancements in science and technology, a strong centralized government, and significant territorial expansion.
    - Other dynasties such as the Han, Song, and Ming also had notable accomplishments.
    - Comparing these dynasties involves evaluating their contributions in various fields.
    - The Tang Dynasty's impact on poetry and culture was profound, but how does this compare t

The concept of the "greatest" dynasty in Chinese history is subjective and depends on the criteria used for evaluation. For the Tang Dynasty, key achievements include the flourishing of poetry, advancements in science and technology, a strong centralized government, and significant territorial expansion. Notable poets like Li Bai and Du Fu thrived during this era, and technological advancements such as the invention of gunpowder and improvements in printing were made. Territorial expansion peaked under Emperor Xuanzong, extending control over parts of Central Asia.

Comparatively, the Han Dynasty contributed significantly to law, philosophy (especially Confucianism), and infrastructure projects like the Grand Canal. The Song Dynasty excelled in science, technology (including the invention of the compass and paper money), and commerce, leading to economic prosperity. The Ming Dynasty is recognized for maritime expeditions led by Admiral Zheng He and major construction projects such as t